In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras.datasets import reuters
from keras.utils import to_categorical
from keras.datasets import boston_housing
from keras import regularizers

In [ ]:
def batchNorm(train_data, test_data):
    
    train = np.copy(train_data)
    test = np.copy(test_data)
    
    mean = train.mean(axis=0)
    train -= mean
    std = train.std(axis=0)
    train /= std

    test -= mean
    test /= std

    return (train, test)


def build_model():
    model = models.Sequential()
    
    model.add(layers.Dense(64, activation='relu', input_shape=(13, )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    
    return model

In [ ]:
# boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

train_data, test_data = batchNorm(train_data, test_data)

k = 3
num_val_samples = len(train_data) // k
num_epochs = 150
all_scores = []
all_mae_history = []

for i in range(k):
    print(f'processing fold: {i}')
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate([train_data[:i * (num_val_samples)], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * (num_val_samples)], train_targets[(i + 1) * num_val_samples:]], axis=0)

    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1)
    mae_history = history.history['mae']
    all_mae_history.append(mae_history)

average_mae_history = [np.mean([x[i] for x in all_mae_history]) for i in range(num_epochs)]

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

In [ ]:
smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)

In [ ]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(f'{test_mae_score=}')